The data is provided courtesy of MAST and the analysis is based on the paper https://iopscience.iop.org/article/10.3847/1538-4357/ad2a50/pdf by Marina Bianchin et al. (2024)

We will start with our imports:

In [1]:
# This is where we import the cubeviz configuration
from jdaviz import Cubeviz

# For accessing the file from MAST and loading it into a temporary directory
import tempfile
from astroquery.mast import Observations
import warnings

# For creating spectral subsets
from glue.core.roi import XRangeROI

# For creating spatial subsets
from glue.core.roi import CircularROI

# For loading line lists
from jdaviz.core.linelists import load_preset_linelist

# For creating a custom line list
from astropy.table import QTable
import astropy.units as u

In [2]:
# Create an instance of cubeviz
cubeviz = Cubeviz()

In [3]:
cubeviz.show('sidecar:right')

In [4]:
fn = 'jw01328-o018_t010_nirspec_g235h-f170lp_s3d.fits'
data_dir = tempfile.gettempdir()

result = Observations.download_file(f"mast:JWST/product/{fn}", local_path=f'{data_dir}/{fn}')

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    cubeviz.load_data(f'{data_dir}/{fn}')

INFO: Found cached file C:\Users\javer\AppData\Local\Temp/jw01328-o018_t010_nirspec_g235h-f170lp_s3d.fits with expected size 198227520. [astroquery.query]


INFO:astroquery:Found cached file C:\Users\javer\AppData\Local\Temp/jw01328-o018_t010_nirspec_g235h-f170lp_s3d.fits with expected size 198227520.


In [5]:
# Retrieve viewer object
spec_viewer = cubeviz.viewers['spectrum-viewer']._obj

# Highlight the H2 1-0S(1) line with a spectral subset
spec_viewer.apply_roi(XRangeROI(2.1544, 2.1588))

In [6]:
# This creates a new spectral subset
spec_viewer.toolbar.active_tool = spec_viewer.toolbar.tools['bqplot:xrange']

# Highlight the Paschen-alpha line with a spectral subset
spec_viewer.apply_roi(XRangeROI(1.9033, 1.9085))

In [7]:
# Load H2 line lists and plot the H 1-0S(1) list
ll = cubeviz.plugins['Line Lists']._obj
ll.open_in_tray()
ll.available_lists
cubeviz.load_line_list(load_preset_linelist('H2'))
cubeviz.erase_spectral_lines()
cubeviz.plot_spectral_line('H 1-0S(1)')

In [8]:
# Add custom line lists to track Paschen-alpha since it is not present in
# available line lists
lt = QTable()
lt['linename'] = ['Paschen-alpha']
lt['rest'] = [1.87513]*u.um
cubeviz.load_line_list(lt)

In [9]:
# Look at statistics for the H2 line
la = cubeviz.plugins['Line Analysis']
la.open_in_tray()
la.spectral_subset = 'Subset 1'
la.continuum = 'Surrounding'

In [10]:
# Select line in order to find redshift of galaxy by comparing rest wavelength
# to the observed wavelength
print(la._obj.line_items)
la._obj.selected_line = 'H 1-0S(1) 2.121'

['H 1-0S(1) 2.121', 'Paschen-alpha 1.87513']


In [11]:
# Estimated redshift
la._obj.selected_line_redshift

0.016875637566430114

In [12]:
# Set line lists to estimated redshift
ll.rs_redshift = la._obj.selected_line_redshift

Now that we have the spectral emissions highlighted, we can create moment maps for each of them to get a better sense of what is happening in the galaxy. We will start with a moment 0 map, which will show us the concentration of the particular element in the galaxy.

In [13]:
# Create a moment map of 0 for the H2 emission line
mm = cubeviz.plugins['Moment Maps']
mm.open_in_tray()
mm.spectral_subset = 'Subset 1'
mm.continuum = 'Surrounding'
mm.n_moment = 0
mm.add_results.label = 'moment-0-h2'
mm.add_results.viewer = 'uncert-viewer'
mm.calculate_moment()

CCDData([[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]], unit='MJy / sr')

In [14]:
# Create a moment map of 0 for the Paschen-alpha emission line
mm = cubeviz.plugins['Moment Maps']
mm.spectral_subset = 'Subset 2'
mm.continuum = 'Surrounding'
mm.n_moment = 0
mm.add_results.label = 'moment-0-pa'
mm.add_results.viewer = 'flux-viewer'
mm.calculate_moment()

CCDData([[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]], unit='MJy / sr')

The paper makes some observations about the increased amounts of Paschen-alpha corresponding to star-forming regions in the north, west, and southern areas of the outer ring. There also appears to be a slight trail of this element going from the nucleus to the ring just south east of the nucleus.

We can also see that there is a high concentration of H2 in the nucleus. The paper asserts that this shows evidence of an AGN, because the high concentration of H2 is consistent with a nuclear radio jet that the authors hypothesize is leading to increased star formation in the galaxy. We will now create moment maps with a moment of 1 to see the flux-weighted centroid of the elements. This will show us the redshifting occuring on in the galaxy.

In [15]:
# Create a moment map of 1 for the H2 emission line
mm = cubeviz.plugins['Moment Maps']
mm.spectral_subset = 'Subset 1'
mm.continuum = 'Surrounding'
mm.n_moment = 1
mm.reference_wavelength = 2.1568
mm.add_results.label = 'moment-1-h2'
mm.add_results.viewer = 'uncert-viewer'
mm.calculate_moment()

CCDData([[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]], unit='km / s')

The velocity dispersion map created from Subset 1 is fairly noisy because the H2 line at that wavelength is weaker. We will instead focus on the Paschen-alpha line at 1.9065 um. 

In [16]:
# Create a moment map of 1 for the Paschen-alpha emission line
mm = cubeviz.plugins['Moment Maps']
mm.spectral_subset = 'Subset 2'
mm.continuum = 'Surrounding'
mm.n_moment = 1
mm.reference_wavelength = 1.9065
mm.add_results.label = 'moment-1-pa'
mm.add_results.viewer = 'flux-viewer'
mm.calculate_moment()

CCDData([[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]], unit='km / s')

In [17]:
# Create a moment map of 2 for the Paschen-alpha emission line
mm = cubeviz.plugins['Moment Maps']
mm.spectral_subset = 'Subset 2'
mm.continuum = 'Surrounding'
mm.n_moment = 2
mm.reference_wavelength = 1.9065
mm.add_results.label = 'moment-2-pa'
mm.add_results.viewer = 'uncert-viewer'
mm.calculate_moment()

CCDData([[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]], unit='km / s')

In [18]:
po = cubeviz.plugins['Plot Options']
po.open_in_tray()
po.viewer = 'flux-viewer'
po.layer = 'moment-1-pa'
po.image_colormap.value = 'rainbow'

In [19]:
po.viewer = 'uncert-viewer'
po.layer = 'moment-2-pa'
po.stretch_preset.value = 90
po.image_colormap.value = 'rainbow'

Positive values in the flux and uncertainty viewers correspond to redshifted areas and negative values correspond to blueshifted areas. This means the galaxy is rotating in a counter-clockwise direction. We can find the minimum and maximum values to get an estimate of the speed of rotation.

For the moment 2 map, we can see the outer ring is relatively calm with values close to the minimum threshold of the detector, while the nucleus is very turbulent. This is another sign of a nuclear radio jet coming from the AGN in the nucleus.

In [21]:
# Create spatial subset around large Paschen-alpha area
flux_viewer = cubeviz.viewers['flux-viewer']._obj
flux_viewer.toolbar.active_tool = flux_viewer.toolbar.tools['bqplot:truecircle']
flux_viewer.apply_roi(CircularROI(xc=28, yc=44.8, radius=5))

In [22]:
# Create spatial subset around the nucleus
flux_viewer.toolbar.active_tool = flux_viewer.toolbar.tools['bqplot:truecircle']'jdaviz:homezoom'
flux_viewer.toolbar.active_tool = flux_viewer.toolbar.tools['bqplot:truecircle']
flux_viewer.apply_roi(CircularROI(xc=23.5, yc=29.2, radius=5))

We can see that Subset 3 is highlighting one of the strongest regions of star formation. Subset 4 is highlighting the strong line flux map in the nucleus. We can use the Spectral Extraction plugin to see how the nucleus looks with a conical aperture.

In [25]:
# We use a conical aperture for the nucleus because it is a PSF (point spread function)
se = cubeviz.plugins['Spectral Extraction']
se.open_in_tray()
se.aperture = 'Subset 4'
se.wavelength_dependent = True
se.aperture_method = 'Exact'
se.function = 'Sum'
se.collapse_to_spectrum()

ValueError: Subset 4 not one of ['Entire Cube', 'Subset 2', 'Subset 3'], reverting selection to Entire Cube